In [1]:
from scraper import WebScraper
from time import sleep

In [2]:
source_cates = dict(top_10_semana = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidGFnX2Jhc2VkX2Nhcm91c2VsIiwidGFnIjoiaG90dGVzdF90aGlzX3dlZWsifQ%3D%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Los%2010%C2%A0m%C3%A1s%20populares%20de%20esta%20semana",

populares_cerca = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidG9wX3N0b3Jlc19ieV9jaXR5X3YyIn0%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Populares%20cerca%20de%20ti",

locales_populares = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidGFnX2Jhc2VkX2Nhcm91c2VsIiwidGFnIjoibW9zdF9wb3B1bGFyX2xvY2FsX3Jlc3RhdXJhbnRzIn0%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Restaurantes%20locales%20m%C3%A1s%20populares",

excelentes_precios = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidGFnX2Jhc2VkX2Nhcm91c2VsIiwidGFnIjoiZ3JlYXRfbWVhbF9ncmVhdF9wcmljZXMifQ%3D%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Buenas%20comidas%20a%20excelentes%20precios")

source_cate = "populares_cerca"

In [3]:
scraper = WebScraper()

scraper.navigate_to_url(source_cates["populares_cerca"])

2024-10-31 12:22:31,400 - INFO - Navegando a la URL: https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidG9wX3N0b3Jlc19ieV9jaXR5X3YyIn0%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Populares%20cerca%20de%20ti


In [4]:
current_address = "Santa ursula coapa" 

In [5]:
# Abrir el menu para gestionar dirección
scraper.click_element('//*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[1]/div/a')

# Entrar al menu para cambiar dirección
scraper.click_element('//*[@id="wrapper"]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[1]/div[2]/a')

# Ingresar dirección
final_address = scraper.enter_address(current_address, 
                                      input_xpath='//*[@id="location-typeahead-location-manager-input"]', 
                                      selection_xpath='//*[@id="location-typeahead-location-manager-item-0"]'
                                      )

2024-10-31 12:22:34,720 - INFO - Elemento en //*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[1]/div/a clickeado correctamente.
2024-10-31 12:22:35,894 - INFO - Elemento en //*[@id="wrapper"]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[1]/div[2]/a clickeado correctamente.
2024-10-31 12:22:39,228 - INFO - Elemento en //*[@id="location-typeahead-location-manager-item-0"] clickeado correctamente.
2024-10-31 12:22:39,229 - INFO - Dirección ingresada correctamente.


In [11]:
# Scrape de tarjetas
sleep(2)
cards_data = scraper.scrape_cards('//*[@id="main-content"]/div/div/div/div/div[3]', 
                                   aditional_info={"cate_source": source_cate,
                                                   "current_address": current_address,
                                                   "final_address": final_address,
                                                   "platform": 'ubereats'
                                                   }
                                   )

2024-10-31 12:24:03,429 - INFO - Scraping completado. Total de tarjetas: 40


In [12]:
cards_data

[{'text': 'La Casa de Toño\n2 Ofertas disponibles\nLa Casa de Toño\nUn restaurante de primera categoría con 4.7 de 5 estrellas de acuerdo con más de 200 calificaciones.\n4.7\nEntrega en 15-30 min\n15–30 min',
  'href': 'https://www.ubereats.com/mx/store/la-casa-de-tono-tlalpan/JPf3GUpYQbmO2qNlzvvNlg',
  'cate_source': 'populares_cerca',
  'current_address': 'Santa ursula coapa',
  'final_address': 'Location marker\nSanta Úrsula Coapa\nCiudad de México, CDMX, México',
  'platform': 'ubereats'},
 {'text': 'Sushi Roll\nAhorros exclusivos\nSushi Roll\nUn restaurante de primera categoría con 4.7 de 5 estrellas de acuerdo con más de 200 calificaciones.\n4.7\nEntrega en 35-50 min\n35–50 min',
  'href': 'https://www.ubereats.com/mx/store/sushi-roll-paseo-acoxpa/8XP_iTybReyI47jsi94lLQ',
  'cate_source': 'populares_cerca',
  'current_address': 'Santa ursula coapa',
  'final_address': 'Location marker\nSanta Úrsula Coapa\nCiudad de México, CDMX, México',
  'platform': 'ubereats'},
 {'text': 'Tort

In [ ]:
scraper.close()